In [3]:
%pip install openai


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [35 lines of output]
           ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/145.4 kB ? eta -:--:--
           ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 143.4/145.4 kB 38.5 MB/s eta 0:00:01
           ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 3.7 MB/s eta 0:00:00
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyprojec

In [12]:
from openai import AzureOpenAI
import os
import json

COMPLETIONS_MODEL = "GPT4-1106"

client = AzureOpenAI(
    azure_endpoint="https://nuevayorkopenaiservice.openai.azure.com/",
    api_key="77c21ef531224c41b2581310f11e4aea",
    api_version="2023-12-01-preview",
)

In [17]:
def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "system", "content": "Enagás es una empresa del sector energético"},
                {"role": "user", "content":"¿Cuál es el sector de Enagás?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_evaluation",
                "description": "Get the evaluation",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "evaluation_score": {
                            "type": "integer",
                            "description": "Score",
                        },
                        "reasoning": {
                            "type": "string",
                            "description": "A reasoning of how the answer was created"},
                        },
                    "required": ["evaluation_score"],
                },
            },
        }
    ]

    response = client.chat.completions.create(
        model=COMPLETIONS_MODEL,
        messages=messages,
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "get_evaluation"}},
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    print(tool_calls)
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        # json_str = tool_calls.split("arguments='")[1].split("', name")[0]
        json_prep = tool_calls[0]
        print(json_prep)
        json_str = json_prep['function']
        dict_args = json.loads(json_str)
        evaluation_score = dict_args['evaluation_score']
        reasoning = dict_args['reasoning']
        print("Evaluation score: " + evaluation_score)
        print("Reasoning: " + reasoning)

    return response_message

In [18]:
print(run_conversation())

[ChatCompletionMessageToolCall(id='call_t55WSOxjlGNMr7BO2qhKx6vP', function=Function(arguments='{"evaluation_score":10,"reasoning":"La respuesta es precisa y correcta. Enagás es una empresa que opera en el sector energ�tico, espec�ficamente en el sector del gas natural. Se encarga del transporte y la gesti�n de la infraestructura del gas y, por tanto, es parte del sector energ�tico."}', name='get_evaluation'), type='function')]


TypeError: 'ChatCompletionMessageToolCall' object is not subscriptable